In [71]:
from pypika import Table, Order, functions
from pypika import MySQLQuery as Query
import mysql.connector
import pandas as pd
import numpy as np
from scipy.ndimage.filters import gaussian_filter
from utils_plotly import exclude_corners

cnx = mysql.connector.connect(
    user='root',
    password='24082408',
    host='localhost',
    database='bsc-explorer'
)

events = Table('events')

query = Query.from_(events).select(
    events.star
)

# .where(
#     events.formationId.isin([105, 106, 107, 108, 109, 110, 111, 112])
# )

events = pd.read_sql(query.get_sql(), cnx)
cnx.close()

events = events.loc[events['IsTouch'] == True]
events = events[['PlayerPosition', 'StartX', 'StartY', 'EndX', 'EndY', 'EventType']]
#events = events.loc[events['PlayerPosition'].isin(['ST'])]
events = exclude_corners(events)
events.shape

(140759, 6)

In [70]:
import plotly.graph_objects as go
import plotly.express as px
from utils_plotly import set_style, draw_field, hex_to_rgba

min_alpha = 0
max_alpha = 0.3

bin_size = 4
n_bins = (26, 17)
x_offset = 0.55

bin_edges_x = [0, 16.5, 34.5, 52.5, 70.5, 88.5, 105]
bin_edges_y = [0, 13.85, 24.85, 43.15, 54.15, 68]

heatmap, xedges, yedges = np.histogram2d(events['StartX'], events['StartY'], bins=(bin_edges_x, bin_edges_y), range=((0, 105), (0, 68)))
heatmap = heatmap.T
#heatmap = (heatmap - heatmap.min())/(heatmap.max()-heatmap.min())
heatmap = (heatmap-heatmap.min())/((heatmap.max()-heatmap.min())/(max_alpha-min_alpha)) + min_alpha

fig = go.Figure()
set_style(fig)
draw_field(fig)

xs = []
for i in range(0, n_bins[1]):
    xs += list(bin_size*np.array(range(0, n_bins[0])) + bin_size/2 + x_offset)

ys = []
for i in range(0, n_bins[1]):
    ys += list([bin_size*i + bin_size/2]*n_bins[0])

k=1
for i in range(0, len(bin_edges_x)-1):
    for j in range(0, len(bin_edges_y)-1):
        x1 = bin_edges_x[i]
        x2 = bin_edges_x[i+1]
        y1 = bin_edges_y[j]
        y2 = bin_edges_y[j+1]

        fig.add_trace(
            go.Scatter(
                x=[x1, x2, x2, x1, x1], y=[y1, y1, y2, y2, y1],
                mode='lines',
                line=dict(
                    color=hex_to_rgba('#8c0b0b', 0.4),
                    width=0,
                ), 
                showlegend=False, 
                fill="tonext", 
                fillcolor=hex_to_rgba('#96783b', heatmap[j][i]),
                hoverinfo='none'
            )
        )

# colors = list(map(lambda x: hex_to_rgba('#8c0b0b', x), heatmap.flatten()))

fig.show()